In [1]:
import numpy as np
import cProfile
from imp import reload

#from info_measures import spatial as ss
import scipy.spatial as ss
import scipy as sp
from info_measures.numpy import kraskov_stoegbauer_grassberger as ksg

In [2]:
sp.__file__

'/home/jesse/Development/scipy/scipy/__init__.py'

# Trees

In [ ]:
x = np.random.randn(10000, 1)

In [ ]:
xtree = ss.cKDTree(x)

In [ ]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=1)

In [ ]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=-1)

In [ ]:
dvec = xtree.query(x, 3+1, p=np.inf, n_jobs=-1)[0][:, 3]

In [10]:
%timeit [len(xtree.query_ball_point(point, 50*dist - 1e-15, p=float('inf'))) for point, dist in zip(x, dvec)]

10 loops, best of 3: 91.9 ms per loop


In [ ]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=-1)]

In [ ]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=1)]

In [6]:
import numpy as np
import scipy.spatial as ss
x = np.random.randn(10000, 1)
xtree = ss.cKDTree(x)

In [7]:
%timeit [xtree.query_ball_point(point, .02, p=np.inf) for point in x]

104 ms ± 576 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit xtree.query_ball_point(x, .02, p=np.inf)

163 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit xtree.count_ball_point(x, .02, p=np.inf)

20 ms ± 479 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit xtree.count_ball_point(x, .02, p=np.inf, n_jobs=4)

15.5 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
r1 = [xtree.query_ball_point(point, .002, p=np.inf) for point in x]
r2 = xtree.query_ball_point(x, .002, p=np.inf)

In [ ]:
all([np.allclose(sorted(rx1), sorted(rx2)) for rx1, rx2 in zip(r1, r2)])

In [4]:
c1 = [len(xtree.query_ball_point(point, .002, p=np.inf)) for point in x]
c2 = xtree.count_ball_point(x, .002, p=np.inf)

In [5]:
print(c1[:10])
print(c2[:10])

[14, 18, 9, 10, 14, 22, 3, 2, 11, 18]
[14 18  9 10 14 22  3  2 11 18]


In [5]:
np.allclose(c1, c2)

False

In [ ]:
%timeit xtree.query_ball_tree(xtree, 2, p=np.inf)

In [ ]:
xtree.query_ball_point(x, .2, p=np.inf)

In [ ]:
assert all([np.allclose(sorted(x), sorted(y)) for x, y in zip(xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=-1),
                                            [xtree.query_ball_point(point, dist - 1e-15, p=float('inf')) for point, dist in zip(x, dvec)])])

In [ ]:
nns1 = xtree.query_ball_point(x, 1., p=np.inf, n_jobs=1)

In [ ]:
nns2 = xtree.query_ball_tree(xtree, 1., p=np.inf)

In [ ]:
all([np.allclose(sorted(x), sorted(y)) for x, y in zip(nns1, nns2)])

In [ ]:
%timeit xtree.query_ball_point(x, 1., p=np.inf, n_jobs=-1)

In [ ]:
%timeit xtree.query_ball_tree(xtree, 1., p=np.inf)

# digamma

In [ ]:
%timeit ksg.avgdigamma1(xtree, x, dvec)
cProfile.run('ksg.avgdigamma1(xtree, x, dvec)')

In [ ]:
%timeit ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)
cProfile.run('ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)')

# MI

In [2]:
import entropy_estimators as ee

In [15]:
x = np.random.randn(50000, 4)

In [16]:
reload(ksg)
reload(ee)
ksg_e = ksg.MutualInformation(x[:,[0, 1]], x[:,[2, 3]])
k=3
dvec = ksg_e.tree.query(ksg_e.Z, k + 1, p=float('inf'))[0][:,k]

In [19]:
ksg_e.mutual_information()

-0.006076569521038522

In [ ]:
[len(xp) for xp in ksg_e.Xtree.query_ball_point(ksg_e.X, dvec)]

In [6]:
%timeit ksg.avgdigamma1(ksg_e.Xtree, ksg_e.X, dvec)

163 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit ksg.avgdigamma1_v(ksg_e.Xtree, ksg_e.X, dvec, n_jobs=1)

34.5 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
ksg_e.mutual_information(v=True)

-0.006076569521038522

In [17]:
%timeit ksg_e.mutual_information(v=True, n_jobs=4)

TypeError: mutual_information() got an unexpected keyword argument 'v'

In [18]:
%timeit ksg_e.mutual_information(v=True, n_jobs=1)

TypeError: mutual_information() got an unexpected keyword argument 'v'

In [13]:
%timeit ksg_e.mutual_information(n_jobs=1)

1.77 s ± 8.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit ee.mi(x[:,[0]], x[:,[1]])

13.1 s ± 40.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
cProfile.run('ksg_e.mutual_information(n_jobs=1)')

         200030 function calls in 0.301 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.301    0.301 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:25(_amax)
        1    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1613(shape)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:163(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2222(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2456(prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:50(_wrapfunc)
        1    0.000    0.000    0.301    0.301 kraskov_stoegbauer_grassberger.py:111(mutual_information)
        2    0.072    0.036    0.289    0.144 kraskov_stoegbauer_grassberger.py:154(avgdigamma1)
        1    0.000    0.000    0.000    0.000 numeric.py:1917(isscalar)
    400

In [30]:
cProfile.run('ksg_e.mutual_information(n_jobs=1, v=True)')

         80114 function calls in 0.089 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.089    0.089 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:25(_amax)
        3    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1613(shape)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:163(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2222(amax)
        3    0.000    0.000    0.000    0.000 fromnumeric.py:2456(prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:50(_wrapfunc)
        4    0.000    0.000    0.000    0.000 index_tricks.py:566(__init__)
        4    0.000    0.000    0.000    0.000 index_tricks.py:574(__iter__)
    40004    0.010    0.000    0.013    0.000 index_tricks.py:585(__next__)
        1    0.000    0.000    0.089    0.089 kraskov